In [1]:
from keras import backend as K

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import keras

from sklearn import preprocessing

from copy import deepcopy
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
style.use('ggplot')
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
# split a multivariate sequence into samples
def split_sequences_day(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences[0])):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences[0])-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[0][i:end_ix].reshape(len(sequences[0][i:end_ix]),1) 
        seq_x = np.append(seq_x, sequences[1][i:end_ix].reshape(len(sequences[1][i:end_ix]),1), axis=1)
        seq_x = np.append(seq_x, sequences[2][i:end_ix].reshape(len(sequences[2][i:end_ix]),1), axis=1)
        seq_y1 =  sequences[0][end_ix]
        X.append(seq_x)
        y.append(seq_y1)
    return np.array(X), np.array(y)

In [4]:
# split a multivariate sequence into samples
def split_sequences_hour(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
       # find the end of this pattern
        end_ix = i + n_steps
#         print(end_ix)
        #check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix]#.reshape(len(sequences[i:end_ix]),1) 
#         print(seq_x)
#         break
#         seq_x = np.append(seq_x, sequences[1][i:end_ix].reshape(len(sequences[1][i:end_ix]),1), axis=1)
#         seq_x = np.append(seq_x, sequences[2][i:end_ix].reshape(len(sequences[2][i:end_ix]),1), axis=1)
        seq_y = sequences[end_ix][-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [5]:
def start_daily_prediction():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.User_Info]# db.db2, db.dn2, db.james, db.leo
    userInfo = []
    for col in collection:
        cursor = col.find({})
        df_user = pd.DataFrame(list(cursor))

    for username in df_user['username']:
        if username == 'pitest' or username == 'jack' or  username == 'manager':
            continue
        collection = [db[username]]
    #     collection = [db.james]# db.db2, db.dn2, db.james, db.leo
        clean_data = []
        for col in collection:
            cursor = col.find({})
            df = pd.DataFrame(list(cursor))

        df.replace('', np.nan, inplace=True)
        df.fillna(method='ffill', inplace=True)

        df['hr_value'] = df['hr_value'].astype(float)
        df['o2_value'] = df['o2_value'].astype(float)
        df['latitude'] = df['latitude'].astype(float)
        df['longitude'] = df['longitude'].astype(float)
        df['step_value'] = df['step_value'].astype(float)
        df = df[(df['hr_value'] != 0)]
        # map out the weekday
        df['weekday'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%w'))
        df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).isocalendar()[1]
                  - (datetime.datetime.fromtimestamp(x).isoweekday() < 7)) # do this so that week may start on monday

        dfsteptotal = pd.DataFrame({})
        dfsteptotal = df[['week', 'weekday']].set_index(['week', 'weekday'])
        dfsteptotal = dfsteptotal[~dfsteptotal.index.duplicated(keep='first')]
        earlestweek = df['week'].values[0]
        latestweek = df['week'].values[-1]
        addindex = pd.DataFrame({})
        for i in range(earlestweek, latestweek):
            for j in range(7):
                if not (i,str(j)) in dfsteptotal.index:
                    add = pd.DataFrame({'week':[i],'weekday':[str(j)]})
                    addindex = addindex.append(add)
        todayweekday = int(datetime.datetime.now().strftime('%w'))
        for j in range(int(todayweekday)):
            if not (latestweek,str(j)) in dfsteptotal.index:
                add = pd.DataFrame({'week':[latestweek],'weekday':[str(j)]})
                addindex = addindex.append(add)
        dfsteptotal.reset_index(inplace=True)
        dfsteptotal = dfsteptotal.append(addindex)
        dfsteptotal = dfsteptotal.set_index(['week', 'weekday'])
        dfsteptotal = dfsteptotal.sort_index()
        dfsteptotal['steps'] = 0

        # here I map between integer and the sting it's mean
        timeslidemap = {0:'morning', 1:'afternoon', 2:'evening'}
        basestamp = df['timestamp'][0]
        endstamp = df['timestamp'][-1:].values[0]
        # for num, row in df[:30000].iterrows():
        while True:
            try:
                dfbetween = df.loc[df['timestamp'].between(
                    datetime.datetime.fromtimestamp(basestamp).replace(hour=0,minute=0,second=0).timestamp(),
                    datetime.datetime.fromtimestamp(basestamp).replace(hour=23,minute=59,second=59).timestamp()
                            , inclusive=True)]
                steps = dfbetween['step_value'].sum()
            #     print(int(datetime.datetime.fromtimestamp(basestamp).strftime('%m')),
            #             int(datetime.datetime.fromtimestamp(basestamp).strftime('%d')),
            #                 timeslidemap[estimedevide])

                dfsteptotal.loc[[(datetime.datetime.fromtimestamp(basestamp).isocalendar()[1] - 
                                            (datetime.datetime.fromtimestamp(basestamp).isoweekday() < 7),
                                    datetime.datetime.fromtimestamp(basestamp).strftime('%w'))], 
                                                   ['steps']] += steps

                basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(days=1)).timestamp()
                if basestamp > endstamp:
                    break
            except Exception as e:
                print(str(e))
                basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(days=1)).timestamp()
        #     else:
        #         print(datetime.datetime.fromtimestamp(basestamp) ,datetime.datetime.fromtimestamp(endstamp))


#         dfsteptotal.loc[(12,'5')] = dfsteptotal.xs('5', level=1).sum() / (dfsteptotal.xs('5', level=1).count() - 1)
        dfsteptotal['Comfort'] = 0
        dfsteptotal['Precp'] = 0

        # add wether 
        coding = 'utf-8'
        table = pd.DataFrame()
        basetime = datetime.datetime.fromtimestamp(df['timestamp'][0])
        num = 0
        for index, row in dfsteptotal.iterrows():
            try:
                # parsing part
                date = basetime + datetime.timedelta(days = num) 
                num += 1
                dateStr = date.strftime('%Y-%m-%d')
                url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0AC60'+ \
                    '&stname=%25E4%25B8%2589%25E5%25B3%25BD&datepicker=' + dateStr
                table = pd.read_html(url,encoding=coding,index_col=0,header=None,flavor='bs4')[1]
                # start dataprocess
                table.replace('/', np.nan, inplace=True)
                table.replace('X', np.nan, inplace=True)  
                table.fillna(method='ffill', inplace=True)
                table.index -= 1
                dfoneday = pd.DataFrame()
                # the table data need to reshape and turn to a variable
                T = table['temperature'].values.reshape(len(table['temperature'].values)).astype('float')
                RH = table['RH'].values.reshape(len(table['RH'].values)).astype('float')
                dfoneday['comfort'] = T - 0.55 *(1-RH/100)*(T - 14)
                dfoneday['Precp'] = table['Precp']['降水量(mm)']
                bins = [0,11,16,20,27,31,60]
                labels=[0,1,2,3,4,5]
                dfoneday['bins'] = pd.cut(dfoneday['comfort'], bins=bins, labels=labels, include_lowest=True)
                # mComfort aComfort eComfort mPrecp aPrecp ePrecp
                dfsteptotal.loc[[index], ['Comfort']] = dfoneday['bins'].astype(int).mean()
                dfoneday['Precp'].astype(float)
                dfsteptotal.loc[[index], ['Precp']] = dfoneday['Precp'].astype(float).sum()
            except Exception as e:
                if str(e) == "could not convert string to float: '...'":
                    print('some data are not yet ready')
                    break
#         print(date)

        steps = dfsteptotal['steps'][:-1].values
        Comfort = dfsteptotal['Comfort'][:-1].values
        Precp = dfsteptotal['Precp'][:-1].values

        scaler = preprocessing.StandardScaler()
        dataset = np.array([steps, Comfort, Precp])

        scaler = preprocessing.StandardScaler()
        dataset = np.array([steps, Comfort, Precp])
        normalized_dataset = scaler.fit_transform(dataset)
        n_steps = 3
        X, y = split_sequences_day(normalized_dataset, n_steps)
        n_features = 3
        model = Sequential()
        model.add(keras.layers.LSTM(100,activation='relu', return_sequences=True,input_shape=(n_steps, n_features)))
        model.add(keras.layers.LSTM(100, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')


        history = model.fit(X, y, epochs=50, verbose=0, batch_size=128, validation_split=0.2)

        newdataset = dataset[:, -3:]
        normalized_dataset = scaler.fit_transform(newdataset.reshape(-1,1))
        normalized_dataset = normalized_dataset.reshape(1,3,3)

        p = model.predict(normalized_dataset)[0]
        prediction = int(scaler.inverse_transform(p))
        date = basetime + datetime.timedelta(days = num - 1)
        dateStr = date.strftime('%Y-%m-%d')
        lis = dateStr.split('-')
        lis = list(map(int, lis))
        
        db = conn.Tracker
        collection = db.prediction# db.db2, db.dn2, db.james, db.leo
        mydict = { 'year':lis[0], 'month':lis[1], 'day':lis[2], 'prediction':prediction, 
                  'type': 'daily',  'user':username}
        collection.insert_one(mydict)
        del model
        conn.close()

In [6]:
def start_hourly_predict():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.User_Info]# db.db2, db.dn2, db.james, db.leo
    userInfo = []
    for col in collection:
        cursor = col.find({})
        df_user = pd.DataFrame(list(cursor))

    for username in df_user['username']:
        if username == 'pitest' or username == 'jack' or username == 'manager':
            continue
        collection = [db[username]]

    # collection = [db.james]# db.db2, db.dn2, db.james, db.leo
        clean_data = []
        for col in collection:
            cursor = col.find({})
            df = pd.DataFrame(list(cursor))
        df.replace('', np.nan, inplace=True)
        df.fillna(method='ffill', inplace=True)

        df['hr_value'] = df['hr_value'].astype(float)
        df['o2_value'] = df['o2_value'].astype(float)
        df['latitude'] = df['latitude'].astype(float)
        df['longitude'] = df['longitude'].astype(float)
        df['step_value'] = df['step_value'].astype(float)

        # map out the weekday
        df['weekday'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%w'))
        df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).isocalendar()[1]
                  - (datetime.datetime.fromtimestamp(x).isoweekday() < 7)) # do this so that week may start on monday

        dfstephour = pd.DataFrame({})
        dfstephour = df[['week', 'weekday']].set_index(['week', 'weekday'])
        dfstephour = dfstephour[~dfstephour.index.duplicated(keep='first')]
        earlestweek = df['week'].values[0]
        latestweek = df['week'].values[-1]
        addindex = pd.DataFrame({})
        for i in range(earlestweek, latestweek):
            for j in range(7):
                if not (i,str(j)) in dfstephour.index:
                    add = pd.DataFrame({'week':[i],'weekday':[str(j)]})
                    addindex = addindex.append(add)
        dfstephour.reset_index(inplace=True)
        dfstephour = dfstephour.append(addindex)
        dfstephour = dfstephour.set_index(['week', 'weekday'])
        dfstephour = dfstephour.sort_index()
        for i in range(24):
            dfstephour[i] = 0

        # here I map between integer and the sting it's mean
        timeslidemap = {0:'msteps', 1:'asteps', 2:'esteps'}
        basestamp = df['timestamp'][0]
        endstamp = df['timestamp'][-1:].values[0]
        # for num, row in df[:30000].iterrows():
        while True:
            try:
                hour = datetime.datetime.fromtimestamp(basestamp).hour
                dfbetween = df.loc[df['timestamp'].between(
                    datetime.datetime.fromtimestamp(basestamp).replace(hour=hour,minute=0,second=0).timestamp(),
                    datetime.datetime.fromtimestamp(basestamp).replace(hour=hour,minute=59,second=59).timestamp()
                            , inclusive=True)]
                steps = dfbetween['step_value'].sum()
            #     print(int(datetime.datetime.fromtimestamp(basestamp).strftime('%m')),
            #             int(datetime.datetime.fromtimestamp(basestamp).strftime('%d')),
            #                 timeslidemap[estimedevide])

                dfstephour.loc[[(datetime.datetime.fromtimestamp(basestamp).isocalendar()[1] - 
                                            (datetime.datetime.fromtimestamp(basestamp).isoweekday() < 7),
                                    datetime.datetime.fromtimestamp(basestamp).strftime('%w'))], 
                                                   [hour]] += steps

                basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(hours=1)).timestamp()
                if basestamp > endstamp:
                    break
            except Exception as e:
        #         print(str(e))
                basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(hours=1)).timestamp()
        #     else:
        #         print(datetime.datetime.fromtimestamp(basestamp) ,datetime.datetime.fromtimestamp(endstamp))
        steps = dfstephour.values
        train_X = []
        for weekday,i in enumerate(steps):
            for hour,j in enumerate(i):
                add = np.array([j])#[weekday, hour, j]
                train_X.append(add)
        # the ending of training data is the  hour of now 
        train_X = train_X[:-(24-datetime.datetime.now().time().hour + 1)]
        train_X = np.array(train_X)

        n_steps = 7
        X, y = split_sequences_hour(train_X, n_steps)

        n_features = 1
        model = Sequential()
        model.add(keras.layers.LSTM(100,activation='relu', return_sequences=True,input_shape=(n_steps, n_features),dropout=0.1))
        model.add(keras.layers.LSTM(50, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        history = model.fit(X, y, epochs=100, verbose=0, batch_size=128, validation_split=0.2)

        p = model.predict(np.array([X[-1]]))
        prediction = int(p)
        futurehour = datetime.datetime.now() + datetime.timedelta(hours=1)
        now = datetime.datetime.now()
        dateStr = now.strftime('%Y-%m-%d')
        lis = dateStr.split('-')
        lis = list(map(int, lis))

        todayweekday = datetime.datetime.now().strftime('%w')
        todayweek = (datetime.datetime.now().isocalendar()[1]
                      - (datetime.datetime.now().isoweekday() < 7))
        print('user', username)
        
        try:
            totalsteps = dfstephour.loc[(todayweek,todayweekday)].values.sum()
        except Exception as e:
            print(str(e))
            totalsteps = 0
        db = conn.Tracker
        collection = db.prediction# db.db2, db.dn2, db.james, db.leo
        mydict = { 'year':lis[0], 'month':lis[1], 'day':lis[2], 'hour':futurehour.hour,'prediction':prediction,
                   'type':'hourly', 'totalsteps':totalsteps, 'user':username}
        collection.insert_one(mydict)
        del model
        conn.close()        

In [11]:
start_hourly_predict()
# start_daily_prediction()

user leo
user james
user shuan
user t
'1'
user jeng


In [9]:
schedule.every().day.at("02:10").do(start_daily_prediction)
schedule.every().hour.at(":20").do(start_hourly_predict)

Every 1 hour at 00:20:00 do start_hourly_predict() (last run: [never], next run: 2019-05-13 19:20:00)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

user leo
user james
user shuan
user t
'1'
user jeng


In [107]:
todayweekday = datetime.datetime.now().strftime('%w')
todayweek = (datetime.datetime.now().isocalendar()[1]
              - (datetime.datetime.now().isoweekday() < 7))
dfstephour.loc[(todayweek,todayweekday)].values.sum()

7837.0